# Notebook to test CausalDA on simulated data

### Imports

In [ ]:
# common libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

# add package path to sys
import sys
sys.path.insert(0, 'c:\\git\\Causal Data Augmentation\\ADMG') # TO CHANGE to match your current folder

# my SCM simulator
from experiments.suite.data.simulations import load_data as causal_data_simu

# from CausalDA
from causal_data_augmentation.api_support.experiments.logging.pickler import Pickler
from causal_data_augmentation.experiment_api import CausalDataAugmentationEagerTrainingExperimentAPI
from causal_data_augmentation.contrib.eager_augmentation_evaluators import PredictionEvaluator, PropertyEvaluator
from causal_data_augmentation.contrib.aug_predictors.xgb import AugXGBRegressor
from causal_data_augmentation.api_support.experiments.logging.pickler import Pickler
import causal_data_augmentation.causal_data_augmentation.api_support.method_config as method_config_module
from ananke.graphs import ADMG
from causal_data_augmentation.causal_data_augmentation.api import EagerCausalDataAugmentation
from causal_data_augmentation.causal_data_augmentation.api import AugmenterConfig, FullAugmentKind

from copy import deepcopy
from typing import List, Tuple, Iterable, Optional, Union
from pathlib import Path
import random

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# evaluation protocol
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_percentage_error
import torch
from sklearn.neighbors import KernelDensity
import scipy
from statsmodels.stats.weightstats import DescrStatsW

## I- Code to call CausalDA

### I-1- Parameters

In [ ]:
### Global Parameters

param_grid_search = {
    'n_estimators': [10, 50, 200],
    'reg_lambda': [1, 10, 100]
}

save_models = False


### CausalDA Parameters

fit_to_aug_only = True

augmenter_config_name = 'FullAugment'
augmenter_config = {
    'normalize_threshold_by_data_size': True,
    'weight_threshold': 1e-2, 
    'weight_threshold_type': 'total',

    'weight_kernel_cfg': {
        'type': 'vanilla_kernel',

        'conti_kertype': 'gaussian', # type of kernel to use for a continuous variable
        'conti_bw_method': 'normal_reference',
        'conti_bw_temperature': 1,

        'ordered_kertype': 'indicator', # type of kernel to use for a discrete and ordered variable
        'ordered_bw_method': 'indicator',

        'unordered_kertype': 'indicator', # type of kernel to use for a discrete and unordered variable
        'unordered_bw_method': 'indicator',

        'const_bandwidth': False, # False: adapt the bandwidth to the type of kerel used
        'bandwidth_temperature': 0.001 # bandwidth used in case of constant bandwidth
    }
}

aug_coeff = [0.5]


### Evaluation parameters

evaluators_param = [(mean_squared_error,'XGB_MSE')]

### I-2- Utils to run and evaluate CausalDA

#### I-2-a Proposed

In [ ]:
def apply_augmenter(augmenter_config: AugmenterConfig, 
                    method: EagerCausalDataAugmentation, 
                    data: pd.DataFrame, 
                    admg: ADMG) -> Tuple[pd.DataFrame, np.ndarray]:
    """Perform the augmentation using the augmenter configured by ``augmenter_config``.

    Parameters:
        augmenter_config : Method configuration.
        method : Instantiated method object.
        data : Data to be augmented.
        admg : ADMG to be used for the augmentation.

    Returns:
        Tuple containing

        - augmented_data : The augmented data DataFrame.
        - aug_weights : The instance weights corresponding to the augmented data.
    """
    if isinstance(augmenter_config, FullAugmentKind):
        augmented_data, aug_weights = method.augment(data, admg)
        aug_weights = aug_weights.flatten()
    else:
        raise NotImplementedError()

    return augmented_data, aug_weights

In [ ]:
def _augment(data: pd.DataFrame, 
             graph, 
             augmenter_config: AugmenterConfig, 
             data_cache_base_path, 
             data_cache_name) -> Tuple[pd.DataFrame, np.ndarray]:
        """Instantiate the method and perform the data augmentation.

        Parameters:
            data : Data to be augmented.
            graph : ADMG to be used for the augmentation.
            augmenter_config : Method configuration.
            data_cache_base_path: The path to the folder to save the trained model and the augmented data
            data_cache_name: The base name that the saved files should follow (it contains the experiment settings)

        Returns:
            Tuple containing

            - augmented_data : The augmented data DataFrame.
            - aug_weights : The instance weights corresponding to the augmented data.
        """
        vertices, di_edges, bi_edges = graph
        admg = ADMG(vertices, di_edges=di_edges, bi_edges=bi_edges)
        method = EagerCausalDataAugmentation(data_cache_base_path, data_cache_name, augmenter_config)

        # Augment
        augmented_data, aug_weights = apply_augmenter(
            augmenter_config, method, data, admg)
        return augmented_data, aug_weights

In [ ]:
def run_method(data: pd.DataFrame, 
               graph,
               predicted_var_name: str,
               predictor_model,
               augmenter_config: AugmenterConfig,
               aug_coeff,
               fit_to_aug_only,
               data_cache_base_path, 
               data_cache_name):
    """Run the method and record the results.

    Parameters:
        data: The data to be augmented.
        graph: The ADMG object used for performing the augmentation.
        predicted_var_name: The name of the predicted variable.
        predictor_model: Trainable predictor model to be trained on the augmented data. Should implement ``fit()`` and ``predict()``.
        augmenter_config: AugmenterConfig,
        aug_coeff: Regularization term in for the augmented data
        fit_to_aug_only: Whether or not to fit the models only to the augmented data
        data_cache_base_path: The path to the folder to save the trained model and the augmented data
        data_cache_name: The base name the saved files should follow (it contains the experiment settings)
    
    Returns:
            List of trained models
    """
    # Augment data
    augmented_data, aug_weights = _augment(data, graph, augmenter_config, data_cache_base_path, data_cache_name)
    
    # Save augmented data and weights 
    augmented_data_to_save_df = augmented_data.copy()
    augmented_data_to_save_df['aug_weights'] = aug_weights
    _augmented_data_pickler = Pickler(data_cache_name + "_augmented", data_cache_base_path)
    _augmented_data_pickler.save(augmented_data_to_save_df)
    
    # self._measure_augmentation(augmented_data, aug_weights, data))

    model_list = []
    predictor = deepcopy(predictor_model)
    for aug_coeff in aug_coeff:
        # Perform training
        if fit_to_aug_only:
            augmented_data = None
            orig_weights = np.zeros(len(data))
        else:
            X = np.array(data.drop(predicted_var_name, axis=1))
            Y = np.array(data[[predicted_var_name]])
            aug_X = np.array(augmented_data.drop(predicted_var_name, axis=1))
            aug_Y = np.array(augmented_data[[predicted_var_name]])
            orig_weights = np.ones(len(data)) / len(data)
            if aug_weights.size > 0:
                orig_weights *= 1 - aug_coeff
                aug_weights *= aug_coeff

        orig_weights *= len(data)
        aug_weights *= len(data)

        predictor.fit(data, augmented_data, orig_weights, aug_weights)
        model_list.append(predictor.model)
    return model_list

In [ ]:
def _evaluate_proposed(dataset, 
                       param_grid, 
                       evaluators_param, 
                       augmenter_config_name, 
                       augmenter_config, 
                       fit_to_aug_only, 
                       aug_coeff, 
                       data_cache_base_path, 
                       data_cache_name, 
                       debug=True,
                       save_models=False):
    """Apply the proposed method, CausalDA, and evaluate it.

    Args:
        dataset (tuple): train_data (dataframe), test_data (dataframe), graph (list of edges), predicted_var_name (string)
        param_grid (dict): {'n_estimators': list of n_estimators to optimize on, 'reg_lambda': list of n_estimators to optimize on}
        evaluators_param (list): list of tuples (eval_metric (function), name (string))
        augmenter_config_name: The name of the type of augmenter
        augmenter_config: AugmenterConfig
        fit_to_aug_only: Whether or not to fit the models only to the augmented data
        aug_coeff: Regularization term in for the augmented data
        data_cache_base_path: The path to the folder to save the trained model and the augmented data
        data_cache_name: The base name the saved files should follow (it contains the experiment settings)
        debug: Whether or not to run the causal_data_augmentation in debug mode printing comments
        save_models: Whether or not to save the trained models

    Returns:
        evaluators_res: liste of the scores given by the evaluators on the models trained with augmentation
    """

    ###################
    ## Get dataset
    ###################
    train_data, test_data, graph, predicted_var_name = dataset
    test_X = test_data.drop(predicted_var_name, axis=1)
    test_Y = test_data[predicted_var_name]
    test_data = (test_X, test_Y)

    ###################
    ## Prepare evaluation
    ###################
    predictor_model = AugXGBRegressor(predicted_var_name, param_grid)
    AugmenterConfigClass = getattr(method_config_module, augmenter_config_name)
    augmenter_config = AugmenterConfigClass(**augmenter_config)

    ###################
    ## Run
    ###################
    model_list = run_method(train_data, 
                            graph,
                            predicted_var_name,
                            predictor_model, 
                            augmenter_config, 
                            aug_coeff,
                            fit_to_aug_only,
                            data_cache_base_path, 
                            data_cache_name)
    # Save XGB models
    if save_models:
        for ind_mod, mod in enumerate(model_list):
            mod.save_model(str(data_cache_base_path) + '/' + str(data_cache_name) + '_xgb_proposed_' + str(ind_mod) + '.json')
    
    ###################
    ## Evaluate
    ###################
    evaluators_res = []
    for mod in model_list:
        mod_res = []
        for ev in evaluators_param:
            pred = mod.predict(test_X)
            mod_res.append((ev[1],ev[0](test_Y,pred)))
        evaluators_res.append(mod_res)
    
    return evaluators_res

#### I-2-b Baseline

In [ ]:
def _evaluate_baseline_xgb(dataset, 
                           param_grid,
                           evaluators_param,
                           data_cache_base_path, 
                           data_cache_name,
                           debug=True,
                           save_models=False):
    """Apply the baseline method, simple XGB, and evaluate it.

    Args:
        dataset (tuple): train_data (dataframe), test_data (dataframe), graph (list of edges), predicted_var_name (string)
        param_grid (dict): {'n_estimators': list of n_estimators to optimize on, 'reg_lambda': list of n_estimators to optimize on}
        evaluators_param (list): list of tuples (eval_metric (function), name (string))
        data_cache_base_path: The path to the folder to save the trained model and the augmented data
        data_cache_name: The base name the saved files should follow (it contains the experiment settings)
        debug: Whether or not to run the causal_data_augmentation in debug mode printing comments
        save_models: Whether or not to save the trained models

    Returns:
        evaluators_res: liste of the scores given by the evaluators on the models trained with augmentation
    """
    train_data, test_data, graph, predicted_var_name = dataset
    test_X = test_data.drop(predicted_var_name, axis=1)
    test_Y = test_data[predicted_var_name]
    test_data = (test_X, test_Y)

    # Prepare evaluation
    predictor_model = AugXGBRegressor(predicted_var_name, param_grid)

    sample_weight = np.ones((len(train_data)))
    predictor_model.fit(train_data, None, sample_weight=sample_weight)
            
    # Save XGB model
    if save_models:
        print(str(data_cache_base_path) + '/' + str(data_cache_name) + '_xgb_baseline.json')
        predictor_model.model.save_model(str(data_cache_base_path) + '/' + str(data_cache_name) + '_xgb_baseline.json')
        
    # Evaluate
    evaluators_res = []
    for ev in evaluators_param:
        pred = predictor_model.predict(test_X)
        evaluators_res.append((ev[1],ev[0](test_Y,pred)))
    
    return evaluators_res

### I-3 Original paper example as a test

In [ ]:
# Paper example : X1 <-- Y --> X2

run_paper_example = False

if run_paper_example:
    vertices = ['V1', 'V2', 'V3']
    di_edges = [('V1', 'V2'), ('V1', 'V3')]
    bi_edges = []

    data_list = []
    for i in range(2):
        for j in range(3*i,3*i+3):
            data_list.append([i,j,j])
    #print(np.array(data_list))
    data = pd.DataFrame(np.array(data_list), columns=vertices)

    predicted_var_name = 'V3'

    data_cache_base_path = ''
    data_cache_base_path = Path(data_cache_base_path)
    data_cache_name = 'simu_test'

    # Intermediate arguments
    AugmenterConfigClass = getattr(method_config_module, augmenter_config_name)
    augmenter_config_ok = AugmenterConfigClass(**augmenter_config)
    method = EagerCausalDataAugmentation(data_cache_base_path, data_cache_name, augmenter_config_ok)

    # admg = ADMG(vertices, di_edges=di_edges, bi_edges=bi_edges)
    # aug_data, aug_weights = apply_augmenter(augmenter_config=augmenter_config_ok, 
    #                                         method=method, 
    #                                         data=data, 
    #                                         admg=admg)

    graph = (vertices, di_edges, bi_edges)
    aug_data, aug_weights = _augment(data, 
                                    graph, 
                                    augmenter_config_ok, 
                                    data_cache_base_path, 
                                    data_cache_name)

    aug_data_to_print = aug_data.copy()
    aug_data_to_print['weight'] = aug_weights*data.shape[0]
    print(aug_data_to_print)
    print(aug_data_to_print['weight'].sum())

## II- Experiments

### II-0- Utils functions

**KL-div, Wasserstein and MMD distances computation from samples**

In [ ]:
def MMD(x, y, kernel):
    """Emprical maximum mean discrepancy. The lower the result, the more evidence that distributions are the same.
    code from : https://www.onurtunali.com/ml/2019/03/08/maximum-mean-discrepancy-in-machine-learning.html

    Args:
        x: first sample, distribution P
        y: second sample, distribution Q
        kernel: kernel type such as "multiscale" or "rbf"
    """
    xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))

    dxx = rx.t() + rx - 2. * xx # Used for A in (1)
    dyy = ry.t() + ry - 2. * yy # Used for B in (1)
    dxy = rx.t() + ry - 2. * zz # Used for C in (1)

    XX, YY, XY = (torch.zeros(xx.shape),
                  torch.zeros(xx.shape),
                  torch.zeros(xx.shape))

    if kernel == "multiscale":
        bandwidth_range = [0.2, 0.5, 0.9, 1.3]
        for a in bandwidth_range:
            XX += a**2 * (a**2 + dxx)**-1
            YY += a**2 * (a**2 + dyy)**-1
            XY += a**2 * (a**2 + dxy)**-1

    if kernel == "rbf":
        bandwidth_range = [10, 15, 20, 50]
        for a in bandwidth_range:
            XX += torch.exp(-0.5*dxx/a)
            YY += torch.exp(-0.5*dyy/a)
            XY += torch.exp(-0.5*dxy/a)
            
    return torch.mean(XX + YY - 2. * XY)


def KL_div(x, y, x_weights=None, y_weights=None, kernel='gaussian'):
    """Emprical Kullback Leibler divergence estimation from Kernel Density Estimates of two distributions defined by their samples.

    Args:
        x: first sample, distribution P
        y: second sample, distribution Q
        kernel: kernel type such as 'gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine'
    """
    # Estimate Kernel Density
    if x_weights is not None:
        kde_x = KernelDensity(kernel=kernel, bandwidth="silverman").fit(x, sample_weight=x_weights)
    else:
        kde_x = KernelDensity(kernel=kernel, bandwidth="silverman").fit(x)
    if y_weights is not None:
        kde_y = KernelDensity(kernel=kernel, bandwidth="silverman").fit(y, sample_weight=y_weights)
    else:
        kde_y = KernelDensity(kernel=kernel, bandwidth="silverman").fit(y)
    
    # Compute density of samples in x and y according to x's law and y's law 
    x_plus_y = np.concatenate((x, y), axis=0)
    p_x = kde_x.score_samples(x_plus_y)
    q_y = kde_y.score_samples(x_plus_y)
    
    # Compute kl div
    kl_div = scipy.stats.entropy(p_x, q_y)
    return kl_div


def wasserstein_distance(x, y, x_weights=None, y_weights=None, kernel='gaussian'):
    """Emprical Wasserstein distance estimation from Kernel Density Estimates of two distributions defined by their samples.

    Args:
        x: first sample, distribution P
        y: second sample, distribution Q
        kernel: kernel type such as 'gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine'
    """
    # Estimate Kernel Density from datasets
    if x_weights is not None:
        kde_x = KernelDensity(kernel=kernel, bandwidth="silverman").fit(x, sample_weight=x_weights)
    else:
        kde_x = KernelDensity(kernel=kernel, bandwidth="silverman").fit(x)
    if y_weights is not None:
        kde_y = KernelDensity(kernel=kernel, bandwidth="silverman").fit(y, sample_weight=y_weights)
    else:
        kde_y = KernelDensity(kernel=kernel, bandwidth="silverman").fit(y)
    
    # Compute density of samples in x and y according to x's law and y's law 
    x_plus_y = np.concatenate((x, y), axis=0)
    p_x = kde_x.score_samples(x_plus_y)
    q_y = kde_y.score_samples(x_plus_y)
    
    # Compute kl div
    w_dist = scipy.stats.wasserstein_distance(p_x, q_y)
    return w_dist

In [ ]:
# Test distances

test_distance_functions = False

if test_distance_functions:
    data = causal_data_simu.generate_data(
            causal_mechanism='nn', #['linear', 'polynomial', 'sigmoid_add', 'sigmoid_mix', 'gp_add', 'gp_mix', 'nn']
            nodes=10,
            expected_degree = 3,
            noise='gaussian', # 'gaussian', 'uniform' or a custom noise function
            noise_coeff=.4,
            dag_type='erdos',
            npoints=1000
        )

    data_x = data.iloc[:500]
    data_y = data.iloc[500:]
    weights_x = np.ones(500)
    weights_y = 0.5*np.ones(500)

    print(KL_div(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian'))
    print(KL_div(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian', x_weights=weights_x, y_weights=weights_x))
    print(KL_div(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian', x_weights=weights_x/500, y_weights=weights_x/500))
    print(KL_div(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian', x_weights=weights_x, y_weights=weights_y))
    print(wasserstein_distance(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian'))
    print(wasserstein_distance(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian', x_weights=weights_x, y_weights=weights_x))
    print(wasserstein_distance(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian', x_weights=weights_x/500, y_weights=weights_x/500))
    print(wasserstein_distance(data_x.to_numpy(), data_y.to_numpy(), kernel='gaussian', x_weights=weights_x, y_weights=weights_y))
    print(float(MMD(torch.tensor(data_x.values), torch.tensor(data_y.values), kernel='rbf').numpy()))

**Augment data and fit XGB function**

In [ ]:
def augment_data_fit_and_evaluate_xgb(data, 
                                      train_size, 
                                      augmenter_config_name, 
                                      augmenter_config, 
                                      graph_,
                                      data_cache_base_path,
                                      data_cache_name,
                                      save_data,
                                      run_results_df,
                                      aggregated_results_df
                                      ):
    ### Build the dataset
    predicted_var_name = data.columns[-1] # Fake varaible to predict --> only to use train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[predicted_var_name]), data[predicted_var_name], train_size=train_size)
    train_data = pd.concat((X_train, y_train), axis=1)
    test_data = pd.concat((X_test, y_test), axis=1)
    
    ### Apply method
    AugmenterConfigClass = getattr(method_config_module, augmenter_config_name)
    augmenter_config_ = AugmenterConfigClass(**augmenter_config)
    aug_data, aug_weights = _augment(train_data, 
                                    graph_, 
                                    augmenter_config_, 
                                    data_cache_base_path, 
                                    data_cache_name)
    
    ### Save data
    if save_data :
        # Augmented data
        augmented_data_to_save_df = aug_data.copy()
        augmented_data_to_save_df['aug_weights'] = aug_weights
        _augmented_data_pickler = Pickler(data_cache_name + "_augmented", data_cache_base_path)
        _augmented_data_pickler.save(augmented_data_to_save_df)
        # Train data
        _train_data_pickler = Pickler(data_cache_name + "_data", data_cache_base_path)
        _train_data_pickler.save(train_data)
        # Test data
        _test_data_pickler = Pickler(data_cache_name + "_test", data_cache_base_path)
        _test_data_pickler.save(test_data)
    
    ### Compute stats on augmented data VS train data : 
    aug_data_df = aug_data.copy()
    aug_data_df['aug_weights'] = aug_weights
    train_df_join_aug = train_data.merge(aug_data_df, how="left", on=list(train_data.columns))
    run_results_df['frac_filtered'] = [train_df_join_aug[train_df_join_aug['aug_weights'].isna()].shape[0] / train_data.shape[0]]
    
    train_data_df = train_data.copy()
    train_data_df['col'] = 1
    aug_df_join_train = aug_data_df.merge(train_data_df, how="left", on=list(aug_data.columns))
    run_results_df['frac_augmented'] = [aug_df_join_train[aug_df_join_train['col'].isna()].shape[0] / train_data.shape[0]]
    
    run_results_df['avg_weight_aug_in_train'] = [aug_df_join_train[aug_df_join_train['col']==1].aug_weights.mean()]
    run_results_df['avg_weight_aug_NOT_in_train'] = [aug_df_join_train[aug_df_join_train['col'].isna()].aug_weights.mean()]
    run_results_df['sum_weight_aug_in_train'] = [aug_df_join_train[aug_df_join_train['col']==1].aug_weights.sum()]
    run_results_df['sum_weight_aug_NOT_in_train'] = [aug_df_join_train[aug_df_join_train['col'].isna()].aug_weights.sum()]
    run_results_df['SUM_aug_weights'] = [aug_data_df['aug_weights'].sum()]
    
    run_results_df['variance_train_NOT_weighted'] = [np.var(train_data.to_numpy(), axis=0)]
    run_results_df['variance_aug_NOT_weighted'] = [np.var(aug_data.to_numpy(), axis=0)]
    run_results_df['variance_aug_weighted'] = [DescrStatsW(aug_data.to_numpy(), weights=aug_weights, ddof=0).var]
    
    if aug_data.shape[0] > 2 : # minimum 10 samples to compute an estimate
        run_results_df['KL_div_train_vs_aug_NOT_weighted'] = [KL_div(aug_data.to_numpy(), train_data.to_numpy(), kernel='gaussian')]
        run_results_df['KL_div_train_vs_aug_weighted'] = [KL_div(aug_data.to_numpy(), train_data.to_numpy(), kernel='gaussian', x_weights=aug_weights)]
        run_results_df['Wasserstein_dist_train_vs_aug_NOT_weighted'] = [wasserstein_distance(aug_data.to_numpy(), train_data.to_numpy(), kernel='gaussian')]
        run_results_df['Wasserstein_dist_train_vs_aug_weighted'] = [wasserstein_distance(aug_data.to_numpy(), train_data.to_numpy(), kernel='gaussian', x_weights=aug_weights)]

    
    ### Evaluate XGB models & Fill results table
    for pred_var in train_data.columns:
        
        # predicted variable caracteristics
        run_results_df['predicted_variable'] = [pred_var]
        if len(list(graph.predecessors(pred_var))) == 0:
            run_results_df['predicted_variable_node_type'] = ['source_node']
            if len(list(graph.successors(pred_var))) == 0:
                run_results_df['predicted_variable_node_type'] = ['isolated_node']
        else:
            if len(list(graph.successors(pred_var))) == 0:
                run_results_df['predicted_variable_node_type'] = ['sink_node']
            else:
                run_results_df['predicted_variable_node_type'] = ['middle_node']
        
        # XGB CausalDA
        if fit_to_aug_only:
            if aug_data.shape[0] > 2 : # minimum 10 samples to compute an estimate
                predictor = AugXGBRegressor(pred_var, param_grid_search)
                predictor.fit(aug_data, None, aug_weights*len(data), np.array([]))
                selected_model_causalDA = predictor.model
        else:
            predictor = AugXGBRegressor(pred_var, param_grid_search)
            orig_weights = (1 - aug_coeff)*np.ones(len(train_data))
            predictor.fit(train_data, aug_data, orig_weights, aug_weights*aug_coeff*len(data))
            selected_model_causalDA = predictor.model
        if aug_data.shape[0] > 2 : # minimum 10 samples to compute an estimate
            run_results_df['MAPE_CausalDA'] = [mean_absolute_percentage_error(test_data[pred_var], selected_model_causalDA.predict(test_data.drop(columns=[pred_var])))]
            run_results_df['r2_CausalDA'] = [r2_score(test_data[pred_var], selected_model_causalDA.predict(test_data.drop(columns=[pred_var])))]
            run_results_df['n_estimators_CausalDA'] = [selected_model_causalDA.n_estimators]
            run_results_df['reg_lambda_CausalDA'] = [selected_model_causalDA.reg_lambda]
        else :
            run_results_df['MAPE_CausalDA'] = [np.nan]
            run_results_df['r2_CausalDA'] = [np.nan]
            run_results_df['n_estimators_CausalDA'] = [np.nan]
            run_results_df['reg_lambda_CausalDA'] = [np.nan]
        
        # Baseline
        predictor = AugXGBRegressor(pred_var, param_grid_search)
        orig_weights = np.ones(len(train_data))
        predictor.fit(train_data, None, orig_weights, np.array([]))
        selected_model_baseline = predictor.model
        run_results_df['n_estimators_Baseline'] = [selected_model_baseline.n_estimators]
        run_results_df['reg_lambda_Baseline'] = [selected_model_baseline.reg_lambda]
        run_results_df['MAPE_Baseline'] = [mean_absolute_percentage_error(test_data[pred_var], selected_model_baseline.predict(test_data.drop(columns=[pred_var])))]
        run_results_df['r2_Baseline'] = [r2_score(test_data[pred_var], selected_model_baseline.predict(test_data.drop(columns=[pred_var])))]                  
    
        # Fill results table
        aggregated_results_df = pd.concat([aggregated_results_df,pd.DataFrame.from_dict(run_results_df)], ignore_index=True)
    return aggregated_results_df

### II-1- Influence des paramètres du problème sur la méthode

##### **Default parameters**

In [ ]:
# chose wether to use light parameters 
use_light_param = True

In [ ]:

### SCM simulation
nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
noise_type = 'gaussian'

### Dataset
train_size = 0.7

### Global Parameters
param_grid_search = {
    'n_estimators': [10, 50, 200],
    'reg_lambda': [1, 10, 100]
}
cv = 3
metric_cv = mean_squared_error
save_models = False

### CausalDA Parameters
augmenter_config_name = 'FullAugment'
augmenter_config = {
    'normalize_threshold_by_data_size': True,
    'weight_threshold': 1e-2, 
    'weight_threshold_type': 'total',
    'weight_kernel_cfg': {
        'type': 'vanilla_kernel',
        'conti_kertype': 'gaussian',
        'conti_bw_method': 'normal_reference',
        'conti_bw_temperature': 1,
        'ordered_kertype': 'indicator',
        'ordered_bw_method': 'indicator',
        'unordered_kertype': 'indicator',
        'unordered_bw_method': 'indicator',
        'const_bandwidth': False,
        'bandwidth_temperature': 0.001
    }
}
fit_to_aug_only = True

### Evaluation parameters
evaluators_param = [(r2_score,'r2'),
                    (mean_absolute_percentage_error, 'MAPE')
                    ]

save_data = False
debug = False

nb_repetition = 20

In [ ]:
columns_agg_res = ['run_id', 
                   'mechanism', 
                   'nb_nodes',
                   'expected_degree',
                   'size_longest_path_in_graph',
                   'noise_coeff',
                   'nb_observations',
                   'predicted_variable',
                   'predicted_variable_node_type',
                   'weight_threshold',
                   'frac_filtered',
                   'frac_augmented',
                   'avg_weight_aug_in_train',
                   'avg_weight_aug_NOT_in_train',
                   'sum_weight_aug_in_train',
                   'sum_weight_aug_NOT_in_train',
                   'SUM_aug_weights',
                   'MAPE_CausalDA',
                   'MAPE_Baseline',
                   'r2_CausalDA',
                   'r2_Baseline',
                   'n_estimators_CausalDA',
                   'n_estimators_Baseline',
                   'reg_lambda_CausalDA',
                   'reg_lambda_Baseline',
                   'KL_div_train_vs_aug_NOT_weighted',
                   'KL_div_train_vs_aug_weighted',
                   'Wasserstein_dist_train_vs_aug_NOT_weighted',
                   'Wasserstein_dist_train_vs_aug_weighted',
                   'variance_train_NOT_weighted',
                   'variance_aug_NOT_weighted',
                   'variance_aug_weighted'
                    ]

#### II-1-a Mechanisms

In [ ]:
mechanism_list = ['linear', 'polynomial', 'sigmoid_mix', 'gp_add', 'gp_mix', 'nn']

In [ ]:
# Light parameters for testing

if use_light_param:
    mechanism_list = ['linear', 'polynomial']
    nb_obs = 100
    nb_var = 5
    expected_degree = 2
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1

In [ ]:
aggregated_results_mechanisms = pd.DataFrame(columns = columns_agg_res)


for mech_i in tqdm(range(len(mechanism_list))):
    mech = mechanism_list[mech_i]
    print(mech)
    for i in range(nb_repetition):
        print("!!NEW!! :Repetition " + str(i) + " on mechanism " + mech)
        run_results_mechanisms = {}
        
        ### run_id and name
        data_cache_base_path = 'mechanisms_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = mech + '_run_' + str(i)
        run_results_mechanisms['run_id'] = [i]
        run_results_mechanisms['mechanism'] = [mech]
        run_results_mechanisms['weight_threshold'] = [augmenter_config['weight_threshold']]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=mech,
                nodes=nb_var,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=expected_degree,
                dag_type='erdos'
            )
        run_results_mechanisms['nb_nodes'] = [nb_var]
        run_results_mechanisms['noise_coeff'] = [noise_coeff]
        run_results_mechanisms['expected_degree'] = expected_degree
        run_results_mechanisms['nb_observations'] = [nb_obs]
        graph = causal_data_simu.load_graph("graph.gpickle")
        run_results_mechanisms['size_longest_path_in_graph'] = [len(nx.dag_longest_path(graph))]
        graph_ = (data.columns,list(graph.edges),[])
        
        ### Augment data, fit and evaluate XGB
        aggregated_results_mechanisms = augment_data_fit_and_evaluate_xgb(data=data,
                                                                          train_size=train_size,
                                                                          augmenter_config_name=augmenter_config_name,
                                                                          augmenter_config=augmenter_config,
                                                                          graph_=graph_,
                                                                          data_cache_base_path=data_cache_base_path,
                                                                          data_cache_name=data_cache_name,
                                                                          save_data=save_data,
                                                                          run_results_df=run_results_mechanisms,
                                                                          aggregated_results_df=aggregated_results_mechanisms)
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_mechanisms_res_' + mech + '.csv'
    aggregated_results_mechanisms.to_csv(path_to_save_csv, index=False)

# save table
path_to_save_csv = str(data_cache_base_path) + '/mechanisms_res.csv'
aggregated_results_mechanisms.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20

#### II-1-b Problem dimensions

In [ ]:
dimensions_list = [7, 8, 9, 10, 15, 20, 25]

In [ ]:
# Light parameters for testing

if use_light_param:
    nb_obs = 10
    causal_mechanism = 'linear'
    expected_degree = 2
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1
    dimensions_list = [5, 6]

In [ ]:
aggregated_results_dimensions = pd.DataFrame(columns = columns_agg_res)


for dim_i in tqdm(range(len(dimensions_list))):
    dim = dimensions_list[dim_i]
    for i in range(nb_repetition):
        print("!!NEW!! :Repetition " + str(i) + " on dimension " + str(dim))
        run_results_dimensions= {}
        
        ### run_id and name
        data_cache_base_path = 'dimension_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = mech + '_run_' + str(i)
        run_results_dimensions['run_id'] = [i]
        run_results_dimensions['mechanism'] = [causal_mechanism]
        run_results_dimensions['weight_threshold'] = [augmenter_config['weight_threshold']]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=causal_mechanism,
                nodes=dim,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=expected_degree,
                dag_type='erdos'
            )
        run_results_dimensions['nb_nodes'] = [dim]
        run_results_dimensions['noise_coeff'] = [noise_coeff]
        run_results_dimensions['expected_degree'] = [expected_degree]
        run_results_dimensions['nb_observations'] = [nb_obs]
        graph = causal_data_simu.load_graph("graph.gpickle")
        run_results_dimensions['size_longest_path_in_graph'] = [len(nx.dag_longest_path(graph))]
        graph_ = (data.columns,list(graph.edges),[])
        
        ### Augment data, fit and evaluate XGB
        aggregated_results_dimensions = augment_data_fit_and_evaluate_xgb(data=data,
                                                                          train_size=train_size,
                                                                          augmenter_config_name=augmenter_config_name,
                                                                          augmenter_config=augmenter_config,
                                                                          graph_=graph_,
                                                                          data_cache_base_path=data_cache_base_path,
                                                                          data_cache_name=data_cache_name,
                                                                          save_data=save_data,
                                                                          run_results_df=run_results_dimensions,
                                                                          aggregated_results_df=aggregated_results_dimensions)
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_dimensions_res_' + str(dim) + '.csv'
    aggregated_results_dimensions.to_csv(path_to_save_csv, index=False)

# save table
path_to_save_csv = str(data_cache_base_path) + '/dimensions_res.csv'
aggregated_results_dimensions.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20

#### II-1-c Graph density - expected degree

In [ ]:
nb_var = 15
density_list = [0, 1, 2, 3, 4, 5, 6, 7]

In [ ]:
# Light parameters for testing

if use_light_param:
    nb_obs = 10
    nb_var = 10
    density_list = [3, 4]
    causal_mechanism = 'linear'
    expected_degree = 2
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1

In [ ]:
aggregated_results_graph_density = pd.DataFrame(columns = columns_agg_res)


for density_i in tqdm(range(len(density_list))):
    density = density_list[density_i]
    for i in range(nb_repetition):
        print("!!NEW!! :Repetition " + str(i) + " on density " + str(density))
        run_results_graph_density= {}
        
        ### run_id and name
        data_cache_base_path = 'graph_density_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = mech + '_run_' + str(i)
        run_results_graph_density['run_id'] = [i]
        run_results_graph_density['mechanism'] = [causal_mechanism]
        run_results_graph_density['weight_threshold'] = [augmenter_config['weight_threshold']]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=causal_mechanism,
                nodes=nb_var,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=density,
                dag_type='erdos'
            )
        run_results_graph_density['nb_nodes'] = [nb_var]
        run_results_graph_density['noise_coeff'] = [noise_coeff]
        run_results_graph_density['expected_degree'] = [density]
        run_results_graph_density['nb_observations'] = [nb_obs]
        graph = causal_data_simu.load_graph("graph.gpickle")
        run_results_graph_density['size_longest_path_in_graph'] = [len(nx.dag_longest_path(graph))]
        graph_ = (data.columns,list(graph.edges),[])
        
        ### Augment data, fit and evaluate XGB
        aggregated_results_graph_density = augment_data_fit_and_evaluate_xgb(data=data,
                                                                          train_size=train_size,
                                                                          augmenter_config_name=augmenter_config_name,
                                                                          augmenter_config=augmenter_config,
                                                                          graph_=graph_,
                                                                          data_cache_base_path=data_cache_base_path,
                                                                          data_cache_name=data_cache_name,
                                                                          save_data=save_data,
                                                                          run_results_df=run_results_graph_density,
                                                                          aggregated_results_df=aggregated_results_graph_density)    
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_graph_density_res_' + str(density) + '.csv'
    aggregated_results_graph_density.to_csv(path_to_save_csv, index=False)

# save full table
path_to_save_csv = str(data_cache_base_path) + '/graph_density_res.csv'
aggregated_results_graph_density.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20

#### II-1-d SCM noise

In [ ]:
noise_coeff_list = [0.1, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
# Light parameters for testing

if use_light_param:
    nb_obs = 10
    nb_var = 5
    causal_mechanism = 'linear'
    expected_degree = 2
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1
    noise_coeff_list = [0.1, 0.2]

In [ ]:
aggregated_results_SCM_noise = pd.DataFrame(columns = columns_agg_res)

for noise_coeff_i in tqdm(range(len(noise_coeff_list))):
    noise_coeff = noise_coeff_list[noise_coeff_i]
    for i in range(nb_repetition):
        print("!!NEW!! :Repetition " + str(i) + " on noise_coeff " + str(noise_coeff))
        run_results_SCM_noise= {}
        
        ### run_id and name
        data_cache_base_path = 'noise_coeff_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = mech + '_run_' + str(i)
        run_results_SCM_noise['run_id'] = [i]
        run_results_SCM_noise['mechanism'] = [causal_mechanism]
        run_results_SCM_noise['weight_threshold'] = [augmenter_config['weight_threshold']]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=causal_mechanism,
                nodes=nb_var,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=expected_degree,
                dag_type='erdos'
            )
        run_results_SCM_noise['nb_nodes'] = [nb_var]
        run_results_SCM_noise['noise_coeff'] = [noise_coeff]
        run_results_SCM_noise['expected_degree'] = [expected_degree]
        run_results_SCM_noise['nb_observations'] = [nb_obs]
        graph = causal_data_simu.load_graph("graph.gpickle")
        run_results_SCM_noise['size_longest_path_in_graph'] = [len(nx.dag_longest_path(graph))]
        graph_ = (data.columns,list(graph.edges),[])
        
        ### Augment data, fit and evaluate XGB
        aggregated_results_SCM_noise = augment_data_fit_and_evaluate_xgb(data=data,
                                                                          train_size=train_size,
                                                                          augmenter_config_name=augmenter_config_name,
                                                                          augmenter_config=augmenter_config,
                                                                          graph_=graph_,
                                                                          data_cache_base_path=data_cache_base_path,
                                                                          data_cache_name=data_cache_name,
                                                                          save_data=save_data,
                                                                          run_results_df=run_results_SCM_noise,
                                                                          aggregated_results_df=aggregated_results_SCM_noise)    
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_noise_coeff_res_' + str(noise_coeff) + '.csv'
    aggregated_results_SCM_noise.to_csv(path_to_save_csv, index=False)

# save full table
path_to_save_csv = str(data_cache_base_path) + '/noise_coeff_res.csv'
aggregated_results_SCM_noise.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20

#### II-1-e Dataset size

In [ ]:
nb_obs_list = [30] + [i for i in range(40, 100, 20)] + [i for i in range(100, 800, 200)]
print(nb_obs_list)

In [ ]:
# Light parameters for testing

if use_light_param:
    nb_obs_list = [10, 15]
    nb_var = 5
    causal_mechanism = 'linear'
    expected_degree = 2
    noise_coeff = 0.4
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1

In [ ]:
aggregated_results_nb_obs = pd.DataFrame(columns = columns_agg_res)

for nb_obs_i in tqdm(range(len(nb_obs_list))):
    nb_obs = nb_obs_list[nb_obs_i]
    for i in range(nb_repetition):
        print("!!NEW!! :Repetition " + str(i) + " on nb_obs " + str(nb_obs))
        run_results_nb_obs= {}
        
        ### run_id and name
        data_cache_base_path = 'nb_observations_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = mech + '_run_' + str(i)
        run_results_nb_obs['run_id'] = [i]
        run_results_nb_obs['mechanism'] = [causal_mechanism]
        run_results_nb_obs['weight_threshold'] = [augmenter_config['weight_threshold']]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=causal_mechanism,
                nodes=nb_var,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=expected_degree,
                dag_type='erdos'
            )
        run_results_nb_obs['nb_nodes'] = [nb_var]
        run_results_nb_obs['noise_coeff'] = [noise_coeff]
        run_results_nb_obs['expected_degree'] = [expected_degree]
        run_results_nb_obs['nb_observations'] = [nb_obs]
        graph = causal_data_simu.load_graph("graph.gpickle")
        run_results_nb_obs['size_longest_path_in_graph'] = [len(nx.dag_longest_path(graph))]
        graph_ = (data.columns,list(graph.edges),[])
        
        ### Augment data, fit and evaluate XGB
        aggregated_results_nb_obs = augment_data_fit_and_evaluate_xgb(data=data,
                                                                          train_size=train_size,
                                                                          augmenter_config_name=augmenter_config_name,
                                                                          augmenter_config=augmenter_config,
                                                                          graph_=graph_,
                                                                          data_cache_base_path=data_cache_base_path,
                                                                          data_cache_name=data_cache_name,
                                                                          save_data=save_data,
                                                                          run_results_df=run_results_nb_obs,
                                                                          aggregated_results_df=aggregated_results_nb_obs)    
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_nb_observations_res_' + str(nb_obs) + '.csv'
    aggregated_results_nb_obs.to_csv(path_to_save_csv, index=False)

# save full table
path_to_save_csv = str(data_cache_base_path) + '/nb_observations_res.csv'
aggregated_results_nb_obs.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20

#### II-1-f Weight threshold

In [ ]:
threshold_list = [1/(10**i) for i in range(1, 6)]
print(threshold_list)

In [ ]:
# Light parameters for testing

if use_light_param:
    threshold_list = [1/(10**i) for i in range(2, 4)]
    nb_var = 5
    causal_mechanism = 'linear'
    expected_degree = 2
    noise_coeff = 0.4
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1

In [ ]:
aggregated_results_threshold = pd.DataFrame(columns = columns_agg_res)

for threshold_i in tqdm(range(len(threshold_list))):
    threshold = threshold_list[threshold_i]
    for i in range(nb_repetition):
        print("!!NEW!! :Repetition " + str(i) + " on threshold " + str(threshold))
        run_results_threshold= {}
        
        augmenter_config['weight_threshold'] = threshold
        run_results_threshold['weight_threshold'] = [threshold]
        
        ### run_id and name
        data_cache_base_path = 'threshold_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = mech + '_run_' + str(i)
        run_results_threshold['run_id'] = [i]
        run_results_threshold['mechanism'] = [causal_mechanism]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=causal_mechanism,
                nodes=nb_var,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=expected_degree,
                dag_type='erdos'
            )
        run_results_threshold['nb_nodes'] = [nb_var]
        run_results_threshold['noise_coeff'] = [noise_coeff]
        run_results_threshold['expected_degree'] = [density]
        run_results_threshold['nb_observations'] = [nb_obs]
        graph = causal_data_simu.load_graph("graph.gpickle")
        run_results_threshold['size_longest_path_in_graph'] = [len(nx.dag_longest_path(graph))]
        graph_ = (data.columns,list(graph.edges),[])
        
        ### Augment data, fit and evaluate XGB
        aggregated_results_threshold = augment_data_fit_and_evaluate_xgb(data=data,
                                                                          train_size=train_size,
                                                                          augmenter_config_name=augmenter_config_name,
                                                                          augmenter_config=augmenter_config,
                                                                          graph_=graph_,
                                                                          data_cache_base_path=data_cache_base_path,
                                                                          data_cache_name=data_cache_name,
                                                                          save_data=save_data,
                                                                          run_results_df=run_results_threshold,
                                                                          aggregated_results_df=aggregated_results_threshold)    
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_threshold_res_' + str(threshold) + '.csv'
    aggregated_results_threshold.to_csv(path_to_save_csv, index=False)

# save full table
path_to_save_csv = str(data_cache_base_path) + '/threshold_res.csv'
aggregated_results_threshold.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20

### II-2- Outliers

In [ ]:
columns_agg_res_outliers = ['run_id', 
                                'predicted_variable',
                                'predicted_variable_node_type',
                                
                                'frac_outliers',
                                'frac_outliers_filtered',
                                
                                'frac_filtered_true',
                                'frac_augmented_true',
                                'avg_weight_aug_in_train_true',
                                'avg_weight_aug_NOT_in_train_true',
                                'SUM_aug_weights_true',
                                'frac_filtered_wrong',
                                'frac_augmented_wrong',
                                'avg_weight_aug_in_train_wrong',
                                'avg_weight_aug_NOT_in_train_wrong',
                                'SUM_aug_weights_wrong',
                                
                                'MAPE_CausalDA_true',
                                'MAPE_CausalDA_wrong',
                                'MAPE_Baseline',
                                'r2_CausalDA_true',
                                'r2_CausalDA_wrong',
                                'r2_Baseline',
                                'n_estimators_CausalDA_true',
                                'n_estimators_CausalDA_wrong',
                                'n_estimators_Baseline',
                                'reg_lambda_CausalDA_true',
                                'reg_lambda_CausalDA_wrong',
                                'reg_lambda_Baseline',
                                
                                'KL_div_train_vs_aug_weighted_true',
                                'KL_div_train_vs_aug_weighted_wrong',
                                'KL_div_aug_true_vs_aug_weighted_wrong',
                                'Wasserstein_dist_train_vs_aug_weighted_true',
                                'Wasserstein_dist_train_vs_aug_weighted_wrong',
                                'Wasserstein_dist_aug_true_vs_aug_weighted_wrong',
                                'variance_train_NOT_weighted',
                                'variance_aug_weighted_true',
                                'variance_aug_weighted_wrong'                 
                                    ]

In [ ]:
frac_outliers_list = [i/100 for i in range(1, 5)] + [i/100 for i in range(5, 20, 5)]
print(frac_outliers_list)

In [ ]:
# Light parameters for testing

if use_light_param:
    nb_var = 5
    frac_outliers_list = [0.2, 0.3]
    causal_mechanism = 'linear'
    noise_type = 'gaussian'
    nb_obs=10
    expected_degree = 2
    noise_coeff = 0.4
    param_grid_search = {
        'n_estimators': [5, 10],
        'reg_lambda': [1, 10]
    }
    cv = 2
    save_data = False
    nb_repetition = 1

In [ ]:
aggregated_results_outliers = pd.DataFrame(columns = columns_agg_res_outliers)

for frac_outliers_i in tqdm(range(len(frac_outliers_list))):
    frac_outliers = frac_outliers_list[frac_outliers_i]
    for i in range(nb_repetition):
        print('Repetition ' + str(i) + ' frac_outliers ' + str(frac_outliers))
        run_results_frac_outliers= {}
        run_results_frac_outliers['frac_outliers'] = [frac_outliers]
        
        ### run_id and name
        data_cache_base_path = 'outliers_variation'
        data_cache_base_path = Path(data_cache_base_path)
        data_cache_name = str(frac_outliers) + '_run_' + str(i)
        run_results_frac_outliers['run_id'] = [i]
        
        ### SCM creation & Data generation
        data = causal_data_simu.generate_data(
                causal_mechanism=causal_mechanism,
                nodes=nb_var,
                noise=noise_type,
                noise_coeff=noise_coeff,
                npoints=nb_obs,
                expected_degree=expected_degree,
                dag_type='erdos'
            )
        graph = causal_data_simu.load_graph("graph.gpickle")
        graph_ = (data.columns,graph.edges,[])
        
        ### Build the dataset
        predicted_var_name = data.columns[-1] # Fake varaible to predict --> only to use train_test_split
        X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[predicted_var_name]), data[predicted_var_name], train_size=train_size)
        train_data = pd.concat((X_train, y_train), axis=1)
        test_data = pd.concat((X_test, y_test), axis=1)
        
        # Randomly add uniformly distributed outliers
        train_data_wrong, outliers_index_list = causal_data_simu.add_uniform_outliers(train_data, frac_outlier=frac_outliers)
        outliers_df = train_data_wrong.iloc[outliers_index_list]
        
        ### Augment data TRUE
        AugmenterConfigClass = getattr(method_config_module, augmenter_config_name)
        augmenter_config_ = AugmenterConfigClass(**augmenter_config)
        aug_data_true, aug_weights_true = _augment(train_data, 
                                                    graph_, 
                                                    augmenter_config_, 
                                                    data_cache_base_path, 
                                                    data_cache_name)
        
        ### Augment data WRONG
        aug_data_wrong, aug_weights_wrong = _augment(train_data_wrong, 
                                                    graph_, 
                                                    augmenter_config_, 
                                                    data_cache_base_path, 
                                                    data_cache_name)

        ### Compute stats on augmented TRUE data VS train data : 
        aug_data_df_true = aug_data_true.copy()
        aug_data_df_true['aug_weights'] = aug_weights_true
        train_df_join_aug_true = train_data.merge(aug_data_df_true, how="left", on=list(train_data.columns))
        run_results_frac_outliers['frac_filtered_true'] = [train_df_join_aug_true[train_df_join_aug_true['aug_weights'].isna()].shape[0] / train_data.shape[0]]
        
        train_data_df = train_data.copy()
        train_data_df['col'] = 1
        aug_df_true_join_train = aug_data_df_true.merge(train_data_df, how="left", on=list(aug_data_true.columns))
        run_results_frac_outliers['frac_augmented_true'] = [aug_df_true_join_train[aug_df_true_join_train['col'].isna()].shape[0] / train_data.shape[0]]
        
        run_results_frac_outliers['avg_weight_aug_in_train_true'] = [aug_df_true_join_train[aug_df_true_join_train['col']==1].aug_weights.mean()]
        run_results_frac_outliers['avg_weight_aug_NOT_in_train_true'] = [aug_df_true_join_train[aug_df_true_join_train['col'].isna()].aug_weights.mean()]
        run_results_frac_outliers['SUM_aug_weights_true'] = [aug_data_df_true['aug_weights'].sum()]
        
        run_results_frac_outliers['variance_train_NOT_weighted'] = [np.var(train_data.to_numpy(), axis=0)]
        run_results_frac_outliers['variance_aug_weighted_true'] = [DescrStatsW(aug_data_true.to_numpy(), weights=aug_weights_true, ddof=0).var]
        
        if aug_data_true.shape[0] > 2 : # minimum 2 samples to compute an estimate
            run_results_frac_outliers['KL_div_train_vs_aug_weighted_true'] = [KL_div(aug_data_true.to_numpy(), train_data.to_numpy(), kernel='gaussian', x_weights=aug_weights_true)]
            run_results_frac_outliers['Wasserstein_dist_train_vs_aug_weighted_true'] = [wasserstein_distance(aug_data_true.to_numpy(), train_data.to_numpy(), kernel='gaussian', x_weights=aug_weights_true)]
        
        ### Compute stats on augmented WRONG data VS train data : 
        aug_data_df_wrong = aug_data_wrong.copy()
        aug_data_df_wrong['aug_weights'] = aug_weights_wrong
        train_df_join_aug_wrong = train_data.merge(aug_data_df_wrong, how="left", on=list(train_data.columns))
        run_results_frac_outliers['frac_filtered_wrong'] = [train_df_join_aug_wrong[train_df_join_aug_wrong['aug_weights'].isna()].shape[0] / train_data.shape[0]]
        
        outliers_df_join_aug_wrong = outliers_df.merge(aug_data_df_wrong, how="left", on=list(outliers_df.columns))
        run_results_frac_outliers['frac_outliers_filtered'] = [outliers_df_join_aug_wrong[outliers_df_join_aug_wrong['aug_weights'].isna()].shape[0] / outliers_df.shape[0]]
        
        train_data_df = train_data.copy()
        train_data_df['col'] = 1
        aug_df_wrong_join_train = aug_data_df_wrong.merge(train_data_df, how="left", on=list(aug_data_wrong.columns))
        run_results_frac_outliers['frac_augmented_wrong'] = [aug_df_wrong_join_train[aug_df_wrong_join_train['col'].isna()].shape[0] / train_data.shape[0]]
        
        run_results_frac_outliers['avg_weight_aug_in_train_wrong'] = [aug_df_wrong_join_train[aug_df_wrong_join_train['col']==1].aug_weights.mean()]
        run_results_frac_outliers['avg_weight_aug_NOT_in_train_wrong'] = [aug_df_wrong_join_train[aug_df_wrong_join_train['col'].isna()].aug_weights.mean()]
        run_results_frac_outliers['SUM_aug_weights_wrong'] = [aug_data_df_wrong['aug_weights'].sum()]

        run_results_frac_outliers['variance_aug_weighted_wrong'] = [DescrStatsW(aug_data_wrong.to_numpy(), weights=aug_weights_wrong, ddof=0).var]
        
        if aug_data_wrong.shape[0] > 2 : # minimum 2 samples to compute an estimate
            run_results_frac_outliers['KL_div_train_vs_aug_weighted_wrong'] = [KL_div(aug_data_wrong.to_numpy(), train_data.to_numpy(), kernel='gaussian', x_weights=aug_weights_wrong)]
            run_results_frac_outliers['Wasserstein_dist_train_vs_aug_weighted_wrong'] = [wasserstein_distance(aug_data_wrong.to_numpy(), train_data.to_numpy(), kernel='gaussian', x_weights=aug_weights_wrong)]

        ### Compute stats on augmented TRUE VS WRONG
        if (aug_data_true.shape[0] > 2) & (aug_data_wrong.shape[0] > 2) : # minimum 2 samples to compute an estimate
            run_results_frac_outliers['KL_div_aug_true_vs_aug_weighted_wrong'] = [KL_div(aug_data_wrong.to_numpy(), 
                                                                                              aug_data_true.to_numpy(), 
                                                                                              kernel='gaussian', 
                                                                                              x_weights=aug_weights_wrong, 
                                                                                              y_weights=aug_weights_true)]
            run_results_frac_outliers['Wasserstein_dist_aug_true_vs_aug_weighted_wrong'] = [wasserstein_distance(aug_data_wrong.to_numpy(), 
                                                                                                                      aug_data_true.to_numpy(), 
                                                                                                                      kernel='gaussian', 
                                                                                                                      x_weights=aug_weights_wrong, 
                                                                                                                      y_weights=aug_weights_true)]
        
        ### Fit XGB models
        for pred_var in train_data.columns:
        
            # predicted variable caracteristics
            run_results_frac_outliers['predicted_variable'] = [pred_var]
            if len(list(graph.predecessors(pred_var))) == 0:
                run_results_frac_outliers['predicted_variable_node_type'] = ['source_node']
                if len(list(graph.successors(pred_var))) == 0:
                    run_results_frac_outliers['predicted_variable_node_type'] = ['isolated_node']
            else:
                if len(list(graph.successors(pred_var))) == 0:
                    run_results_frac_outliers['predicted_variable_node_type'] = ['sink_node']
                else:
                    run_results_frac_outliers['predicted_variable_node_type'] = ['middle_node']
            
            # XGB CausalDA TRUE
            if fit_to_aug_only:
                if aug_data_true.shape[0] > 2 : # minimum 2 samples to compute an estimate
                    predictor = AugXGBRegressor(pred_var, param_grid_search)
                    predictor.fit(aug_data_true, None, aug_weights_true*len(train_data), np.array([]))
                    selected_model_causalDA_true = predictor.model
            else:
                predictor = AugXGBRegressor(pred_var, param_grid_search)
                orig_weights = (1 - aug_coeff)*np.ones(len(train_data))
                predictor.fit(train_data, aug_data_true, orig_weights, aug_weights_true*aug_coeff*len(train_data))
                selected_model_causalDA_true = predictor.model
            if aug_data_true.shape[0] > 2 : # minimum 2 samples to compute an estimate
                run_results_frac_outliers['MAPE_CausalDA_true'] = [mean_absolute_percentage_error(test_data[pred_var], selected_model_causalDA_true.predict(test_data.drop(columns=[pred_var])))]
                run_results_frac_outliers['r2_CausalDA_true'] = [r2_score(test_data[pred_var], selected_model_causalDA_true.predict(test_data.drop(columns=[pred_var])))]
                run_results_frac_outliers['n_estimators_CausalDA_true'] = [selected_model_causalDA_true.n_estimators]
                run_results_frac_outliers['reg_lambda_CausalDA_true'] = [selected_model_causalDA_true.reg_lambda]
            else :
                run_results_frac_outliers['MAPE_CausalDA_true'] = [np.nan]
                run_results_frac_outliers['r2_CausalDA_true'] = [np.nan]
                run_results_frac_outliers['n_estimators_CausalDA_true'] = [np.nan]
                run_results_frac_outliers['reg_lambda_CausalDA_true'] = [np.nan]
            
            # XGB CausalDA WRONG
            if fit_to_aug_only:
                if aug_data_wrong.shape[0] > 2 : # minimum 2 samples to compute an estimate
                    predictor = AugXGBRegressor(pred_var, param_grid_search)
                    predictor.fit(aug_data_wrong, None, aug_weights_wrong*len(train_data), np.array([]))
                    selected_model_causalDA_wrong = predictor.model
            else:
                predictor = AugXGBRegressor(pred_var, param_grid_search)
                orig_weights = (1 - aug_coeff)*np.ones(len(train_data))
                predictor.fit(train_data, aug_data_wrong, orig_weights, aug_weights_wrong*aug_coeff*len(train_data))
                selected_model_causalDA_wrong = predictor.model
            if aug_data_wrong.shape[0] > 2 : # minimum 2 samples to compute an estimate
                run_results_frac_outliers['MAPE_CausalDA_wrong'] = [mean_absolute_percentage_error(test_data[pred_var], selected_model_causalDA_wrong.predict(test_data.drop(columns=[pred_var])))]
                run_results_frac_outliers['r2_CausalDA_wrong'] = [r2_score(test_data[pred_var], selected_model_causalDA_wrong.predict(test_data.drop(columns=[pred_var])))]
                run_results_frac_outliers['n_estimators_CausalDA_wrong'] = [selected_model_causalDA_wrong.n_estimators]
                run_results_frac_outliers['reg_lambda_CausalDA_wrong'] = [selected_model_causalDA_wrong.reg_lambda]
            else :
                run_results_frac_outliers['MAPE_CausalDA_wrong'] = [np.nan]
                run_results_frac_outliers['r2_CausalDA_wrong'] = [np.nan]
                run_results_frac_outliers['n_estimators_CausalDA_wrong'] = [np.nan]
                run_results_frac_outliers['reg_lambda_CausalDA_wrong'] = [np.nan]
            
            # Baseline
            predictor = AugXGBRegressor(pred_var, param_grid_search)
            orig_weights = np.ones(len(train_data))
            predictor.fit(train_data, None, orig_weights, np.array([]))
            selected_model_baseline = predictor.model
            run_results_frac_outliers['n_estimators_Baseline'] = [selected_model_baseline.n_estimators]
            run_results_frac_outliers['reg_lambda_Baseline'] = [selected_model_baseline.reg_lambda]
            run_results_frac_outliers['MAPE_Baseline'] = [mean_absolute_percentage_error(test_data[pred_var], selected_model_baseline.predict(test_data.drop(columns=[pred_var])))]
            run_results_frac_outliers['r2_Baseline'] = [r2_score(test_data[pred_var], selected_model_baseline.predict(test_data.drop(columns=[pred_var])))]                  
        
            # Fill results table
            aggregated_results_outliers = pd.concat([aggregated_results_outliers,pd.DataFrame.from_dict(run_results_frac_outliers)], ignore_index=True)
        
    
    # save intermediate results
    path_to_save_csv = str(data_cache_base_path) + '/intermediate_outliers_res_' + str(frac_outliers) + '.csv'
    aggregated_results_outliers.to_csv(path_to_save_csv, index=False)

# save full table
path_to_save_csv = str(data_cache_base_path) + '/outliers_res.csv'
aggregated_results_outliers.to_csv(path_to_save_csv, index=False)

In [ ]:

# Reset default parameters

nb_obs = 500
causal_mechanism = 'nn'
nb_var = 10
expected_degree = 3
noise_coeff = 0.4
augmenter_config['weight_threshold'] = 1e-2
nb_repetition = 20